In [1]:
from bert_functions_v2 import analyzeWord, getBert
from createDims import createPolarDimension
from get_data_from_file import create_lookup_from_data_file,create_lookupFiles_out_of_adjectives_list_using_file
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
import pickle
import json
import string
import ast
import torch
import pandas as pd
import re
from collections import defaultdict

In [2]:

def get_name(antonym):
    return wn.synset(antonym).lemma_names()[0]


def get_examples(antonym):
    antonym= wn.synset(antonym)
    examples=antonym.examples()
    #save only examples that containt the required word
    correct_examples=[]
    for example in examples:
        if re.search(r'\b'+str(antonym.lemma_names()[0])+'\\b', example, re.I) is not None:
            correct_examples.append(example)
    

    
    examples = [sent.translate(str.maketrans({k: " " for k in string.punctuation})) for sent in correct_examples]
    # add a space after each sentence
    return ['{} '.format(sent) for sent in examples]



def create_lookup_files(antonyms, lookup_path):
    if len(np.unique(antonyms, axis=0)) != len(antonyms):
        print("Your antonym list contains duplicates. Please try again!")
        return
    
    # get all word sense definitions
    synset_defs = [[wn.synset(anto).definition() for anto in pair] for pair in antonyms]
    # get example sentences from wordnet
    examples_readable = {str(pair):{get_name(anto): get_examples(anto) for anto in pair} for pair in antonyms}
    examples_lookup = [[[get_name(anto), get_examples(anto)] for anto in pair] for pair in antonyms]
    
    # save 
    with open(out_path + 'lookup_synset_dict.txt', 'w') as t:
        t.write(json.dumps(antonyms, indent=4))
    with open(out_path + 'lookup_synset_dict.pkl', 'wb') as p:
        pickle.dump(antonyms, p)
    with open(lookup_path + 'lookup_synset_definition.txt', 'w') as t:
        t.write(json.dumps(synset_defs, indent=4))  
    with open(lookup_path + 'lookup_synset_definition.pkl', 'wb') as p:
        pickle.dump(synset_defs, p)        
    with open(lookup_path + 'antonym_wordnet_example_sentences_readable_extended.txt', 'w') as t:
        t.write(json.dumps(examples_readable, indent=4))  
    with open(lookup_path + 'lookup_anto_example_dict.txt', 'w') as t:
        t.write(json.dumps(examples_lookup, indent=4))      
    with open(lookup_path + 'lookup_anto_example_dict.pkl', 'wb') as p:
        pickle.dump(examples_lookup, p)
    return



def create_dimensions_list():
    from collections import defaultdict
    word="1"
    #d for dimensions
    d=[]
    while word != '' :
        antonyms=defaultdict()
        index=0
        word=input('Enter word: ')
        if word !='':
            #get all synstets
            synsets=wn.synsets(word)

            #create dictionary only with synsets that have an antonym
            for i in synsets:
                if(len(i.lemmas()[0].antonyms()) !=0):
                    ant=i.lemmas()[0].antonyms()[0]
                    antonyms[i]=ant.synset()
                    
            #keep only antonym pairs that have examples
            for key in list(antonyms.keys()):
                if (len(key.examples())==0 or len(antonyms[key].examples())==0):
                    del antonyms[key]


            #visualize definitions

            for key in list(antonyms.keys()):
                print("Antonym pair index {}".format(index))
                print("   Antonym pair {}-----{} ".format(key,antonyms[key]))
                print("        Definition {} => {} ".format(key,key.definition()))
                print("        Definition {} => {} ".format(antonyms[key],antonyms[key].definition()))
                print(" ")
                print(" ")
                print(" ")
                index+=1
            
            #create list of desired dimensions 
            selected_antonym_pair_index=int(input('Enter the index of desired antonym pair: '))

            key=list(antonyms.keys())[selected_antonym_pair_index]

            d.append(list([key.name(),antonyms[key].name()]))
            print(" ")
            print(" ")
            print(" ")

    return d



In [3]:
dims=create_dimensions_list()

Enter word: hot
Antonym pair index 0
   Antonym pair Synset('hot.a.01')-----Synset('cold.a.01') 
        Definition Synset('hot.a.01') => used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning 
        Definition Synset('cold.a.01') => having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration 
 
 
 
Antonym pair index 1
   Antonym pair Synset('hot.a.03')-----Synset('cold.a.02') 
        Definition Synset('hot.a.03') => extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm 
        Definition Synset('cold.a.02') => extended meanings; especially of psychological coldness; without human warmth or emotion 
 
 
 
Enter the index of desired antonym pair: 0
 
 
 
Enter word: 


In [4]:
dims

[['hot.a.01', 'cold.a.01']]

In [5]:
out_path = 'antonyms/example/'

create_lookup_files(dims, out_path)

In [6]:
# get the embedding model 
tokenizer, model = getBert()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# create the base change matrix (this might take some time)
createPolarDimension(model, tokenizer, out_path=out_path, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")

Start forwarding the Polar opposites ...


In [8]:
context=["today is a warm day","today is a hot day","today is a very hot day","today is an extremely hot day","today is the hottest day ever",
        "today is so hot that water evaporates immediately","today is so hot, you can cook an egg in asphalt","today is extremely hot, you can cook an egg in asphalt"
        ,"today you can cook an egg in asphalt","today the temperatures are not high","today the temperatures are high",
        "today the temperatures are normal","today the temperatures are low","today the temperatures are really high",
        "today the temperatures are extremely high","today the temperature is 10 degrees Celsius","today the temperature is 15 degrees Celsius",
        "today the temperature is 20 degrees Celsius","today the temperature is 30 degrees Celsius","today the temperature is 40 degrees Celsius",
        "today the temperature is ten degrees Celsius","today the temperature is fifteen degrees Celsius",
        "today the temperature is twenty degrees Celsius","today the temperature is thirty degrees Celsius","today the temperature is fourty degrees Celsius",
        "today the temperatures are below zero","today is cold","today I feel like I am freezing"]

In [9]:


# base change does not work well with only few dimensions -> compare with projection
antonym_path = out_path + "polar_dimensions.pkl"
word = "today"
results_hot_cold=defaultdict()
for sent in context:

    res=analyzeWord(word, sent, model=model,tokenizer=tokenizer, antonym_path=antonym_path, lookup_path=out_path, numberPolar=1,method="projection")
    results_hot_cold[sent]=float(list(res[2].values())[0])


Analyzing the word:  today
In the context of:  today is a warm day
Top:  1
Dimension:  hot<------>cold
Definitions:  used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning<------>having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration
Value: -0.32459828


Analyzing the word:  today
In the context of:  today is a hot day
Top:  1
Dimension:  hot<------>cold
Definitions:  used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning<------>having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration
Value: -0.67671514


Analyzing the word:  today
In the context of:  today is a very hot day
Top:  1
Dimension:  hot<------>cold
Definitions:  used of physical heat; having a high or hi

Top:  1
Dimension:  hot<------>cold
Definitions:  used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning<------>having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration
Value: -0.07872139


Analyzing the word:  today
In the context of:  today the temperature is ten degrees Celsius
Top:  1
Dimension:  hot<------>cold
Definitions:  used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning<------>having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration
Value: -0.10618258


Analyzing the word:  today
In the context of:  today the temperature is fifteen degrees Celsius
Top:  1
Dimension:  hot<------>cold
Definitions:  used of physical heat; having a high or higher than desira

In [10]:

dict(sorted(results_hot_cold.items(), key=lambda item: item[1]))


{'today is so hot that water evaporates immediately': -1.0303345918655396,
 'today I feel like I am freezing': -0.9680099487304688,
 'today is so hot, you can cook an egg in asphalt': -0.9650959372520447,
 'today is extremely hot, you can cook an egg in asphalt': -0.929493248462677,
 'today is an extremely hot day': -0.896325409412384,
 'today is a hot day': -0.6767151355743408,
 'today is a very hot day': -0.6473714709281921,
 'today the temperatures are really high': -0.6312922835350037,
 'today the temperatures are not high': -0.4053514003753662,
 'today you can cook an egg in asphalt': -0.38347339630126953,
 'today the temperatures are normal': -0.36136630177497864,
 'today is a warm day': -0.3245982825756073,
 'today the temperatures are high': -0.267612487077713,
 'today the temperatures are extremely high': -0.2038285732269287,
 'today is the hottest day ever': -0.1681911200284958,
 'today the temperatures are below zero': -0.16513758897781372,
 'today the temperatures are low':

In [11]:
wn.synset('hot.a.01').examples()

['hot stove',
 'hot water',
 'a hot August day',
 'a hot stuffy room',
 "she's hot and tired",
 'a hot forehead']

In [12]:
wn.synset('cold.a.01').examples()

['a cold climate',
 'a cold room',
 'dinner has gotten cold',
 'cold fingers',
 'if you are cold, turn up the heat',
 'a cold beer']

In [41]:

out='antonyms_file/' #folder where the lookup files will be saved
file='data_file.xlsx' #name of the file
create_lookup_from_data_file(file,out)

# get the embedding model 
tokenizer, model = getBert()
print("Model imported")

# create the base change matrix (this might take some time)
createPolarDimension(model, tokenizer, out_path=out, antonym_path=out + "antonym_wordnet_example_sentences_readable_extended.txt")
print("dimensions created")


# base change does not work well with only few dimensions -> compare with projection
antonym_path = out + "polar_dimensions.pkl"
word = "today"

context=["today is a warm day","today is a hot day","today is a very hot day","today is an extremely hot day","today is the hottest day ever",
        "today is so hot that water evaporates immediately","today is so hot, you can cook an egg in asphalt","today is extremely hot, you can cook an egg in asphalt"
        ,"today you can cook an egg in asphalt","today the temperatures are not high","today the temperatures are high",
        "today the temperatures are normal","today the temperatures are low","today the temperatures are really high",
        "today the temperatures are extremely high","today the temperature is 10 degrees Celsius","today the temperature is 15 degrees Celsius",
        "today the temperature is 20 degrees Celsius","today the temperature is 30 degrees Celsius","today the temperature is 40 degrees Celsius",
        "today the temperature is ten degrees Celsius","today the temperature is fifteen degrees Celsius",
        "today the temperature is twenty degrees Celsius","today the temperature is thirty degrees Celsius","today the temperature is fourty degrees Celsius",
        "today the temperatures are below zero","today is cold","today I feel like I am freezing"]

results_hot_cold=defaultdict()
for sent in context:

    res=analyzeWord(word, sent, model=model,tokenizer=tokenizer, antonym_path=antonym_path, lookup_path=out, numberPolar=1,method="projection")
    results_hot_cold[sent]=float(list(res[2].values())[0])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model imported
Start forwarding the Polar opposites ...
dimensions created
Analyzing the word:  today
In the context of:  today is a warm day
Top:  1
Dimension:  hot<------>cold
Definitions:   <------> 
Value:                      0.46382177


Analyzing the word:  today
In the context of:  today is a hot day
Top:  1
Dimension:  hot<------>cold
Definitions:   <------> 
Value:                      0.49748722


Analyzing the word:  today
In the context of:  today is a very hot day
Top:  1
Dimension:  hot<------>cold
Definitions:   <------> 
Value:                      0.023824146


Analyzing the word:  today
In the context of:  today is an extremely hot day
Top:  1
Dimension:  hot<------>cold
Definitions:   <------> 
Value: -0.1501015


Analyzing the word:  today
In the context of:  today is the hottest day ever
Top:  1
Dimension:  hot<------>cold
Definitions:   <------> 
Value:                      0.8675604


Analyzing the word:  today
In the context of:  today is so hot that water evap

In [42]:
dict(sorted(results_hot_cold.items(), key=lambda item: item[1]))

{'today you can cook an egg in asphalt': -0.22418707609176636,
 'today the temperatures are not high': -0.1924322247505188,
 'today is an extremely hot day': -0.15010149776935577,
 'today the temperatures are really high': -0.1386573761701584,
 'today the temperatures are extremely high': -0.13640937209129333,
 'today the temperatures are high': -0.12754736840724945,
 'today the temperature is twenty degrees Celsius': -0.0147255165502429,
 'today the temperature is thirty degrees Celsius': 0.004635116085410118,
 'today the temperature is 30 degrees Celsius': 0.004843641072511673,
 'today the temperature is 20 degrees Celsius': 0.007207153365015984,
 'today the temperature is ten degrees Celsius': 0.017539700493216515,
 'today is a very hot day': 0.023824146017432213,
 'today the temperature is 15 degrees Celsius': 0.0242109764367342,
 'today the temperature is 10 degrees Celsius': 0.03737922012805939,
 'today the temperature is 40 degrees Celsius': 0.04256487265229225,
 'today the temp

In [15]:
dims=create_dimensions_list()

Enter word: difficult
Antonym pair index 0
   Antonym pair Synset('difficult.a.01')-----Synset('easy.a.01') 
        Definition Synset('difficult.a.01') => not easy; requiring great physical or mental effort to accomplish or comprehend or endure 
        Definition Synset('easy.a.01') => posing no difficulty; requiring little effort 
 
 
 
Enter the index of desired antonym pair: 0
 
 
 
Enter word: 


In [16]:
out_path = 'antonyms/example/'

create_lookup_files(dims, out_path)

In [17]:
dims

[['difficult.a.01', 'easy.a.01']]

In [18]:
# get the embedding model 
tokenizer, model = getBert()
# create the base change matrix (this might take some time)
createPolarDimension(model, tokenizer, out_path=out_path, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start forwarding the Polar opposites ...


In [26]:
context=["this is not a difficult task",
        "this is a difficult task",
        "this tasks requires not a big effort",
        "this task requires an enormous effort",
        "this task is very difficult",
        "this dask is exceptionaly difficult",
        "this is an easy task",
        "this is a challenging task",
        "this task challanged me so much I have no energy left",
        "for this task I spent 400 hours to finish",
        "this is not a difficult problem",
        "this is not a difficult situation",
        "this is  a difficult situation",
        "this is not a difficult answer",
         "this is  a difficult answer",
        "this is  an easy answer"]

In [27]:


# base change does not work well with only few dimensions -> compare with projection
antonym_path = out_path + "polar_dimensions.pkl"
word = "this"
results_diff_easy=defaultdict()
for sent in context:

    res=analyzeWord(word, sent, model=model,tokenizer=tokenizer, antonym_path=antonym_path, lookup_path=out_path, numberPolar=1,method="projection")
    results_diff_easy[sent]=float(list(res[2].values())[0])


Analyzing the word:  this
In the context of:  this is not a difficult task
Top:  1
Dimension:  difficult<------>easy
Definitions:  not easy; requiring great physical or mental effort to accomplish or comprehend or endure<------>posing no difficulty; requiring little effort
Value: -1.0006989


Analyzing the word:  this
In the context of:  this is a difficult task
Top:  1
Dimension:  difficult<------>easy
Definitions:  not easy; requiring great physical or mental effort to accomplish or comprehend or endure<------>posing no difficulty; requiring little effort
Value: -1.575955


Analyzing the word:  this
In the context of:  this tasks requires not a big effort
Top:  1
Dimension:  difficult<------>easy
Definitions:  not easy; requiring great physical or mental effort to accomplish or comprehend or endure<------>posing no difficulty; requiring little effort
Value: -1.2670445


Analyzing the word:  this
In the context of:  this task requires an enormous effort
Top:  1
Dimension:  difficult<-

In [28]:
wn.synset('difficult.a.01').examples() 

['a difficult task',
 'nesting places on the cliffs are difficult of access',
 'difficult times',
 'why is it so hard for you to keep a secret?']

In [29]:
wn.synset('easy.a.01').examples() 

['an easy job',
 'an easy problem',
 'an easy victory',
 'the house is easy to heat',
 'satisfied with easy answers',
 'took the easy way out of his dilemma']

In [30]:
#the word task tends to give values towards difficult
#does not understand the "not"
dict(sorted(results_diff_easy.items(), key=lambda item: item[1]))


{'this is  a difficult situation': -1.8528460264205933,
 'this task is very difficult': -1.75127375125885,
 'this is a challenging task': -1.69247305393219,
 'this task requires an enormous effort': -1.6463702917099,
 'this is a difficult task': -1.5759550333023071,
 'this dask is exceptionaly difficult': -1.5645759105682373,
 'for this task I spent 400 hours to finish': -1.4893882274627686,
 'this task challanged me so much I have no energy left': -1.4699734449386597,
 'this tasks requires not a big effort': -1.2670445442199707,
 'this is not a difficult task': -1.0006989240646362,
 'this is  a difficult answer': -0.888251006603241,
 'this is an easy task': -0.7701345086097717,
 'this is not a difficult situation': -0.7561586499214172,
 'this is not a difficult problem': -0.401968777179718,
 'this is not a difficult answer': -0.26677170395851135,
 'this is  an easy answer': 0.6096210479736328}

In [43]:

out='antonyms_file/' #folder where the lookup files will be saved
file='data_file.xlsx' #name of the file
create_lookup_from_data_file(file,out)

# get the embedding model 
tokenizer, model = getBert()
print("Model imported")

# create the base change matrix (this might take some time)
createPolarDimension(model, tokenizer, out_path=out, antonym_path=out + "antonym_wordnet_example_sentences_readable_extended.txt")
print("dimensions created")


# base change does not work well with only few dimensions -> compare with projection
antonym_path = out + "polar_dimensions.pkl"
word = "this"

context=["this is not a difficult task",
        "this is a difficult task",
        "this tasks requires not a big effort",
        "this task requires an enormous effort",
        "this task is very difficult",
        "this dask is exceptionaly difficult",
        "this is an easy task",
        "this is a challenging task",
        "this task challanged me so much I have no energy left",
        "for this task I spent 400 hours to finish",
        "this is not a difficult problem",
        "this is not a difficult situation",
        "this is  a difficult situation",
        "this is not a difficult answer",
         "this is  a difficult answer",
        "this is  an easy answer"]

results_diff_easy=defaultdict()
for sent in context:

    res=analyzeWord(word, sent, model=model,tokenizer=tokenizer, antonym_path=antonym_path, lookup_path=out, numberPolar=1,method="projection")
    results_diff_easy[sent]=float(list(res[2].values())[0])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model imported
Start forwarding the Polar opposites ...
dimensions created
Analyzing the word:  this
In the context of:  this is not a difficult task
Top:  1
Dimension:  difficult<------>easy
Definitions:   <------> 
Value: -0.99287504


Analyzing the word:  this
In the context of:  this is a difficult task
Top:  1
Dimension:  difficult<------>easy
Definitions:   <------> 
Value: -1.2480205


Analyzing the word:  this
In the context of:  this tasks requires not a big effort
Top:  1
Dimension:  difficult<------>easy
Definitions:   <------> 
Value: -0.29779056


Analyzing the word:  this
In the context of:  this task requires an enormous effort
Top:  1
Dimension:  difficult<------>easy
Definitions:   <------> 
Value: -1.5612599


Analyzing the word:  this
In the context of:  this task is very difficult
Top:  1
Dimension:  difficult<------>easy
Definitions:   <------> 
Value: -0.6530743


Analyzing the word:  this
In the context of:  this dask is exceptionaly difficult
Top:  1
Dimension: 

In [44]:
dict(sorted(results_diff_easy.items(), key=lambda item: item[1]))

{'this task challanged me so much I have no energy left': -2.8897275924682617,
 'for this task I spent 400 hours to finish': -2.4451191425323486,
 'this is  a difficult situation': -1.5924957990646362,
 'this task requires an enormous effort': -1.5612598657608032,
 'this is  a difficult answer': -1.3992407321929932,
 'this is a challenging task': -1.397113561630249,
 'this is a difficult task': -1.2480205297470093,
 'this dask is exceptionaly difficult': -1.0636757612228394,
 'this is not a difficult task': -0.9928750395774841,
 'this is not a difficult answer': -0.9838048219680786,
 'this is not a difficult situation': -0.9257106184959412,
 'this is an easy task': -0.7671908140182495,
 'this task is very difficult': -0.653074324131012,
 'this is  an easy answer': -0.6395812034606934,
 'this is not a difficult problem': -0.45698603987693787,
 'this tasks requires not a big effort': -0.2977905571460724}

In [5]:
dims=create_dimensions_list()

Enter word: happy
Antonym pair index 0
   Antonym pair Synset('happy.a.01')-----Synset('unhappy.a.01') 
        Definition Synset('happy.a.01') => enjoying or showing or marked by joy or pleasure 
        Definition Synset('unhappy.a.01') => experiencing or marked by or causing sadness or sorrow or discontent 
 
 
 
Enter the index of desired antonym pair: 0
 
 
 
Enter word: 


In [6]:
out_path = 'antonyms/example/'

create_lookup_files(dims, out_path)
# get the embedding model 
tokenizer, model = getBert()
# create the base change matrix (this might take some time)
createPolarDimension(model, tokenizer, out_path=out_path, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start forwarding the Polar opposites ...


In [8]:
context= ["He felt happy and relaxed after a day of fishing on the lake.",
"The sound of waves crashing against the shore made her feel happy and peaceful.",
"She felt happy and grateful for her home and all its comforts.",
"The sight of a field of wildflowers made him feel happy and grateful for nature.",
"She had a big smile on her face as she danced around the room.",
"The children shrieked with joy as they played together.",
"He felt a surge of happiness as he watched the sunset.",
"She laughed until tears streamed down her face.",
"The puppy wagged its tail happily as it played with its new toy.",
"He felt a sense of contentment as he sat on the beach and listened to the waves.",
"She hugged her loved ones tightly and felt a warm sense of happiness.",
"The smell of fresh flowers made her feel happy and at peace.",
"He had a skip in his step as he walked down the street, feeling happy and carefree.",
"The sun shining on her face made her feel happy and alive.",
"She couldn't stop grinning as she read the news of her promotion.",
"The sound of her child's laughter filled her heart with happiness.",
"He felt a sense of accomplishment and happiness after completing a difficult task.",
"The taste of her favorite food made her feel happy and content.",
"She felt a sense of belonging and happiness as she spent time with her friends.",
"The sight of a rainbow made her feel happy and hopeful.",
"Her face was twisted into a deep frown, her eyes filled with tears.",
"He slumped in his seat, shoulders sagging in defeat.",
"She let out a long, heavy sigh as she stared at the floor.",
"His voice was low and bitter as he spoke.",
"She couldn't stop the tears from streaming down her face.",
"He leaned against the wall, eyes closed in frustration.",
"She sat alone in the corner, looking lost and dejected.",
"His hands shook with anger as he clenched his fists.",
"She walked with her head down, shoulders hunched in sadness.",
"His eyes were dull and lifeless, lacking their usual spark."]

In [11]:


# base change does not work well with only few dimensions -> compare with projection
antonym_path = out_path + "polar_dimensions.pkl"

results_happy_unhappy=defaultdict()
for sent in context:
    word = sent.split()[0]
    res=analyzeWord(word, sent, model=model,tokenizer=tokenizer, antonym_path=antonym_path, lookup_path=out_path, numberPolar=1,method="projection")
    results_happy_unhappy[sent]=float(list(res[2].values())[0])


Analyzing the word:  He
In the context of:  He felt happy and relaxed after a day of fishing on the lake.
Top:  1
Dimension:  happy<------>unhappy
Definitions:  enjoying or showing or marked by joy or pleasure<------>experiencing or marked by or causing sadness or sorrow or discontent
Value: -0.1178519


Analyzing the word:  The
In the context of:  The sound of waves crashing against the shore made her feel happy and peaceful.
Top:  1
Dimension:  happy<------>unhappy
Definitions:  enjoying or showing or marked by joy or pleasure<------>experiencing or marked by or causing sadness or sorrow or discontent
Value: -0.80811065


Analyzing the word:  She
In the context of:  She felt happy and grateful for her home and all its comforts.
Top:  1
Dimension:  happy<------>unhappy
Definitions:  enjoying or showing or marked by joy or pleasure<------>experiencing or marked by or causing sadness or sorrow or discontent
Value: -0.61426985


Analyzing the word:  The
In the context of:  The sight of a

Top:  1
Dimension:  happy<------>unhappy
Definitions:  enjoying or showing or marked by joy or pleasure<------>experiencing or marked by or causing sadness or sorrow or discontent
Value:                      1.3996689


Analyzing the word:  His
In the context of:  His hands shook with anger as he clenched his fists.
Top:  1
Dimension:  happy<------>unhappy
Definitions:  enjoying or showing or marked by joy or pleasure<------>experiencing or marked by or causing sadness or sorrow or discontent
Value:                      1.6501318


Analyzing the word:  She
In the context of:  She walked with her head down, shoulders hunched in sadness.
Top:  1
Dimension:  happy<------>unhappy
Definitions:  enjoying or showing or marked by joy or pleasure<------>experiencing or marked by or causing sadness or sorrow or discontent
Value:                      1.2716864


Analyzing the word:  His
In the context of:  His eyes were dull and lifeless, lacking their usual spark.
Top:  1
Dimension:  happy<-----

In [12]:
dict(sorted(results_happy_unhappy.items(), key=lambda item: item[1]))

{"The sound of her child's laughter filled her heart with happiness.": -1.7336193323135376,
 'The sun shining on her face made her feel happy and alive.': -1.3098117113113403,
 'The sound of waves crashing against the shore made her feel happy and peaceful.': -0.8081106543540955,
 'The sight of a rainbow made her feel happy and hopeful.': -0.755073070526123,
 'She had a big smile on her face as she danced around the room.': -0.7497670650482178,
 'The smell of fresh flowers made her feel happy and at peace.': -0.7494044899940491,
 'The children shrieked with joy as they played together.': -0.7057815194129944,
 'The taste of her favorite food made her feel happy and content.': -0.703997015953064,
 'The sight of a field of wildflowers made him feel happy and grateful for nature.': -0.6657332181930542,
 'She felt happy and grateful for her home and all its comforts.': -0.6142698526382446,
 "She couldn't stop grinning as she read the news of her promotion.": -0.5986639261245728,
 "She could

In [13]:
results_happy_unhappy

defaultdict(None,
            {'He felt happy and relaxed after a day of fishing on the lake.': -0.11785189807415009,
             'The sound of waves crashing against the shore made her feel happy and peaceful.': -0.8081106543540955,
             'She felt happy and grateful for her home and all its comforts.': -0.6142698526382446,
             'The sight of a field of wildflowers made him feel happy and grateful for nature.': -0.6657332181930542,
             'She had a big smile on her face as she danced around the room.': -0.7497670650482178,
             'The children shrieked with joy as they played together.': -0.7057815194129944,
             'He felt a surge of happiness as he watched the sunset.': 0.1254727691411972,
             'She laughed until tears streamed down her face.': -0.24761493504047394,
             'The puppy wagged its tail happily as it played with its new toy.': -0.5447828769683838,
             'He felt a sense of contentment as he sat on the beach and lis

In [14]:

out='antonyms_file/' #folder where the lookup files will be saved
file='data_file.xlsx' #name of the file
create_lookup_from_data_file(file,out)

# get the embedding model 
tokenizer, model = getBert()
print("Model imported")

# create the base change matrix (this might take some time)
createPolarDimension(model, tokenizer, out_path=out, antonym_path=out + "antonym_wordnet_example_sentences_readable_extended.txt")
print("dimensions created")


# base change does not work well with only few dimensions -> compare with projection
antonym_path = out + "polar_dimensions.pkl"




results_happy_unhappy=defaultdict()
for sent in context:
    word = sent.split()[0]
    res=analyzeWord(word, sent, model=model,tokenizer=tokenizer, antonym_path=antonym_path, lookup_path=out, numberPolar=1,method="projection")
    results_happy_unhappy[sent]=float(list(res[2].values())[0])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model imported
Start forwarding the Polar opposites ...
dimensions created
Analyzing the word:  He
In the context of:  He felt happy and relaxed after a day of fishing on the lake.
Top:  1
Dimension:  happy<------>unhappy
Definitions:   <------> 
Value: -1.1565646


Analyzing the word:  The
In the context of:  The sound of waves crashing against the shore made her feel happy and peaceful.
Top:  1
Dimension:  happy<------>unhappy
Definitions:   <------> 
Value: -2.5233471


Analyzing the word:  She
In the context of:  She felt happy and grateful for her home and all its comforts.
Top:  1
Dimension:  happy<------>unhappy
Definitions:   <------> 
Value: -1.3140144


Analyzing the word:  The
In the context of:  The sight of a field of wildflowers made him feel happy and grateful for nature.
Top:  1
Dimension:  happy<------>unhappy
Definitions:   <------> 
Value: -2.961357


Analyzing the word:  She
In the context of:  She had a big smile on her face as she danced around the room.
Top:  1
D

In [15]:
dict(sorted(results_happy_unhappy.items(), key=lambda item: item[1]))

{"The sound of her child's laughter filled her heart with happiness.": -3.659740447998047,
 'The sight of a field of wildflowers made him feel happy and grateful for nature.': -2.9613571166992188,
 'The sun shining on her face made her feel happy and alive.': -2.9363536834716797,
 'The sight of a rainbow made her feel happy and hopeful.': -2.9214181900024414,
 'The smell of fresh flowers made her feel happy and at peace.': -2.8992130756378174,
 'The taste of her favorite food made her feel happy and content.': -2.7160801887512207,
 'The sound of waves crashing against the shore made her feel happy and peaceful.': -2.5233471393585205,
 'She had a big smile on her face as she danced around the room.': -2.086466073989868,
 'She hugged her loved ones tightly and felt a warm sense of happiness.': -1.6212053298950195,
 'The children shrieked with joy as they played together.': -1.6198389530181885,
 'The puppy wagged its tail happily as it played with its new toy.': -1.6135698556900024,
 'She

In [16]:
results_happy_unhappy

defaultdict(None,
            {'He felt happy and relaxed after a day of fishing on the lake.': -1.1565645933151245,
             'The sound of waves crashing against the shore made her feel happy and peaceful.': -2.5233471393585205,
             'She felt happy and grateful for her home and all its comforts.': -1.3140144348144531,
             'The sight of a field of wildflowers made him feel happy and grateful for nature.': -2.9613571166992188,
             'She had a big smile on her face as she danced around the room.': -2.086466073989868,
             'The children shrieked with joy as they played together.': -1.6198389530181885,
             'He felt a surge of happiness as he watched the sunset.': -1.0663968324661255,
             'She laughed until tears streamed down her face.': -0.19837766885757446,
             'The puppy wagged its tail happily as it played with its new toy.': -1.6135698556900024,
             'He felt a sense of contentment as he sat on the beach and list